In [2]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
to_fit = pd.read_csv("Data/costs_input_new.csv")

In [4]:
to_fit.head()

,Variable Name,Value,Lower Bound,Upper Bound
0,cLevel1NurseryAdmissionPerDay,703.0,234.00,1172.00
1,cLevel1NursingCarePerDay,410.9,273.93,684.83
2,cLevel2NICUAdmissionPerDay,3066.0,1022.00,5109.00
3,cLevel2NursingCarePerDay,501.1,334.07,835.17
4,cGentamicin1,21.0,7.00,34.00


In [5]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_normal_distr(value, lower_bound, upper_bound):
    std_err_left = (value - lower_bound)/1.96
    std_err_right = (upper_bound - value)/1.96
    return value, max(std_err_left, std_err_right)

In [6]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_gamma_distr(value, lower_bound, upper_bound):

    scale_lower_bound = np.around((abs(lower_bound - value)/2)**2 / value, 2)
    scale_upper_bound = scale_lower_bound + 10
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    scale_lower_bound = int((((upper_bound - value)/1.8)**2 / value) / 100) * 100
    scale_upper_bound = scale_lower_bound + 100000
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    print("No match found")
    return 0, 0

In [7]:
to_fit["mu"], to_fit["sigma"] = np.vectorize(fit_normal_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

In [8]:
to_fit

,Variable Name,Value,Lower Bound,Upper Bound,mu,sigma
0,cLevel1NurseryAdmissionPerDay,703.0,234.00,1172.00,703.0,2.392857e+02
1,cLevel1NursingCarePerDay,410.9,273.93,684.83,410.9,1.397602e+02
2,cLevel2NICUAdmissionPerDay,3066.0,1022.00,5109.00,3066.0,1.042857e+03
3,cLevel2NursingCarePerDay,501.1,334.07,835.17,501.1,1.704439e+02
4,cGentamicin1,21.0,7.00,34.00,21.0,7.142857e+00
5,cAmpicillin1,60.0,20.00,100.00,60.0,2.040816e+01
6,cInitialNICUTests,682.0,227.00,1137.00,682.0,2.321429e+02
7,cBloodCulture1,172.0,57.00,287.00,172.0,5.867347e+01
8,cCSF,104.0,35.00,174.00,104.0,3.571429e+01
9,cReadmission,5378.5,4302.80,6454.19,5378.5,5.488265e+02


In [9]:
to_fit.to_csv("Data/costs_output_normal.csv")